In [1]:
import os
import argparse
from datetime import datetime
import glob
import numpy as np
from numpy import linalg
from matplotlib import pyplot as plt
import pandas as pd

In [35]:
iou_threshold = 0.2

In [2]:
def iou(x1, y1, w1, h1, x2, y2, w2, h2):
  # convert the coordinates to the top-left and bottom-right corners
  x1_1 = x1 - w1 / 2
  y1_1 = y1 - h1 / 2
  x1_2 = x1 + w1 / 2
  y1_2 = y1 + h1 / 2
  x2_1 = x2 - w2 / 2
  y2_1 = y2 - h2 / 2
  x2_2 = x2 + w2 / 2
  y2_2 = y2 + h2 / 2

  # compute the area of the bounding boxes
  area1 = w1 * h1
  area2 = w2 * h2

  # compute the intersection area
  xi1 = max(x1_1, x2_1)
  yi1 = max(y1_1, y2_1)
  xi2 = min(x1_2, x2_2)
  yi2 = min(y1_2, y2_2)
  inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

  # compute the union area
  union_area = area1 + area2 - inter_area

  # compute the IoU
  iou = inter_area / union_area

  return iou

In [36]:
gt_txt = '/Users/andreaberti/Documents/work/DBT/test_inference/valid/labels_gt/DBT-P01839_DBT-S03748_lcc_mass0_slice18.txt'
pred_txt = '/Users/andreaberti/Documents/work/DBT/test_inference/valid/labels_detection/DBT-P01839_DBT-S03748_lcc_mass0_slice18.txt'

In [37]:
gt_df = pd.read_csv(gt_txt, sep=' ', header=None)

gt_list = []
pred_list = []

In [38]:
for idx in gt_df.index:
            x_center = gt_df.iloc[idx][1]
            y_center = gt_df.iloc[idx][2]
            width = gt_df.iloc[idx][3]
            height = gt_df.iloc[idx][4]

            gt_element = [x_center, y_center, width, height]
            gt_list.append(gt_element) # è una lista di liste, ciascuna delle quali contiene gli elementi di un bbox


In [46]:
found_masses = np.zeros(shape=len(gt_list))

In [40]:
pred_df = pd.read_csv(pred_txt, sep=' ', header=None)


for idx in pred_df.index:
    x_center = pred_df.iloc[idx][1]
    y_center = pred_df.iloc[idx][2]
    width = pred_df.iloc[idx][3]
    height = pred_df.iloc[idx][4]
    confidence = pred_df.iloc[idx][5]

    pred_element = [x_center, y_center, width, height, confidence]
    pred_list.append(pred_element)


In [41]:
gt_list

[[0.7840681362725451,
  0.4334554334554334,
  0.1703406813627254,
  0.2332112332112332]]

In [42]:
pred_list

[[0.720942, 0.330484, 0.0551102, 0.03663, 0.103345],
 [0.785321, 0.399064, 0.0736473, 0.0630851, 0.107638]]

In [47]:
for detection in pred_list:
    distances = []
    # per ogni elemento trovato, inizializzo una lista, 
    # poi guardo quanto dista da tutti i bbox della gt e appendo alla lista
    for bbox in gt_list:
        pred_xy = np.array(detection[0:2])
        gt_xy = np.array(bbox[0:2])

        distance = linalg.norm(pred_xy - gt_xy)
        distances.append(distance)

    # trovo a quale massa GT sta più vicino e per quella calcolo la IOU
    closest_item = np.argmin(distances)
    gt_values = gt_list[closest_item]
    iou_value = iou(x1=detection[0], y1=detection[1], w1=detection[2], h1=detection[3],
                    x2=gt_values[0], y2=gt_values[1], w2=gt_values[2], h2=gt_values[3])

    # se la IOU è minore di una certa soglia, le considero come coincidenti
    if iou_value >= iou_threshold:
        # print(f'è un vero Vero Positivo, massa più vicina riga {closest_item}')
        found_masses[closest_item] = 1

In [44]:
detection = pred_list[1]
distances = []
# per ogni elemento trovato, inizializzo una lista, 
# poi guardo quanto dista da tutti i bbox della gt e appendo alla lista
for bbox in gt_list:
    pred_xy = np.array(detection[0:2])
    gt_xy = np.array(bbox[0:2])

    distance = linalg.norm(pred_xy - gt_xy)
    distances.append(distance)

# trovo a quale massa GT sta più vicino e per quella calcolo la IOU
closest_item = np.argmin(distances)
gt_values = gt_list[closest_item]
iou_value = iou(x1=detection[0], y1=detection[1], w1=detection[2], h1=detection[3],
                x2=gt_values[0], y2=gt_values[1], w2=gt_values[2], h2=gt_values[3])

# se la IOU è minore di una certa soglia, le considero come coincidenti
if iou_value >= iou_threshold:
    # print(f'è un vero Vero Positivo, massa più vicina riga {closest_item}')
    found_masses[closest_item] = 1

In [48]:
found_masses

array([0.])